In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%cd "/content/drive/MyDrive/LLaMaprac/llama"

/content/drive/MyDrive/LLaMaprac/llama


In [2]:
!pip install -r requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.7 MB/s eta 0:00:00


In [6]:
!pip uninstall peft -y -q
!pip install -q git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.6 MB/s eta 0:00:00


## Instruction 데이터 셋 만들기

In [7]:
import numpy as np
import pandas as pd
import operator
import os
import string
import re
import random
import sys
import platform
import math
import time
import datetime
import json

from collections import Counter, defaultdict

from matplotlib import rcParams, pyplot as plt
import seaborn as sns

import sklearn as sk
from sklearn.utils import check_random_state
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

import nltk
from nltk.corpus import stopwords, wordnet
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import ne_chunk, tree2conlltags
from nltk.probability import FreqDist

nltk.download('stopwords')
eng_stopwords = set(stopwords.words("english"))
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')
symbols_knowns = string.ascii_letters + string.digits + string.punctuation

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore')

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
train = pd.read_csv('/content/drive/MyDrive/Judgement_dataset/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Judgement_dataset/test.csv')
print("Number of rows in train dataset : ", train.shape[0])

Number of rows in train dataset :  2478


In [9]:
# train 데이터프레임에서 facts 열의 각 행(row)에 대해 문장 수를 세는 함수를 정의합니다.
def count_sentences(row):
    sentences = sent_tokenize(row)
    return len(sentences)

# facts 열의 문장 수를 세어서 새로운 'num_sentences' 열을 추가합니다.
train['num_sentences'] = train['facts'].apply(lambda x: count_sentences(x))

# 문장 수가 1개인 행(row)들만 선택합니다.
rows_with_single_sentence = train[train['num_sentences'] == 1]

# 결과를 DataFrame으로 출력합니다.
rows_with_single_sentence

,ID,first_party,second_party,facts,first_party_winner,num_sentences
248,TRAIN_0248,New York,Ferber,A New York child pornography law prohibited pe...,1,1
554,TRAIN_0554,Riverside County,McLaughlin,McLaughlin was arrested without a warrant and ...,1,1
630,TRAIN_0630,Earl R. Deen,"Gulf, Colorado & Santa Fe Railway Company",Not Available.\n,1,1
764,TRAIN_0764,Nollan,California Coastal Commission,The California Coastal Commission required own...,1,1
858,TRAIN_0858,Morrison,Olson,The Ethics in Government Act of 1978 created a...,1,1
1104,TRAIN_1104,Eastern Enterprises,Apfel,Currently unknown.\n,1,1
1285,TRAIN_1285,Smith,Daily Mail Publishing Company,A West Virginia statute made it a crime for a ...,0,1
1319,TRAIN_1319,National Association for the Advancement of Co...,Button,The NAACP was prosecuted for violating a Virgi...,1,1
1329,TRAIN_1329,United States,Paradise,In response to a series of NAACP-initiated law...,0,1
1461,TRAIN_1461,Jones,Alfred H. Mayer Company,"Jones, a black man, charged that a real estate...",1,1


In [10]:
# 특정 인덱스 삭제 - Not available, Currently available, Currently unknown
indexes_to_remove = [630, 1104, 1595]
train = train.drop(indexes_to_remove)
train = train.reset_index(drop=True)

In [11]:
# 'facts' 열에서 'Not available'이나 'Currently available' 문구가 있는지 확인
is_not_available = train['facts'].str.contains('Not available', case=False)
is_currently_available = train['facts'].str.contains('Currently available', case=False)

In [12]:
train['facts'].duplicated().value_counts()

False    2475
Name: facts, dtype: int64

In [13]:
train.drop("ID", axis=1, inplace=True)

train

,first_party,second_party,facts,first_party_winner,num_sentences
0,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1,7
1,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0,7
2,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1,8
3,Linkletter,Walker,Victor Linkletter was convicted in state court...,0,3
4,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1,9
...,...,...,...,...,...
2470,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",Congress amended the Clean Air Act through the...,1,5
2471,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.","Alliance Bond Fund, Inc., an investment fund, ...",1,7
2472,Peguero,United States,"In 1992, the District Court sentenced Manuel D...",0,6
2473,Immigration and Naturalization Service,St. Cyr,"On March 8, 1996, Enrico St. Cyr, a lawful per...",0,8


### Data Argument

텍스트 분류를 위해서 Data Augment화를 하는 테크닉<br>
SR : Synonym Replacement (동의어 대체)<br>
RD : Random Deletion (랜덤 삭제)<br>
RS : Random Swap (랜덤 스왑)<br>
RI : Random Insertion (랜덤 삽입)<br>


In [17]:
# SR

def get_synonyms(word):
  synonyms = set() # 동의어 집합 생성

  for syn in wordnet.synsets(word):
    for l in syn.lemmas():
      synonym = l.name().replace("_", " ").replace("-", " ").lower()
      # "_", "-"는 모두 공백으로 대체하고, 소문자형으로 모두 교체한다.
      synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])

      synonyms.add(synonym) # 추출된 동의어는 이제 동의어 집합에 집어넣는다.

  # 만약 단어가 동의어에 포함되어있으면, 그 단어는 삭제한다.
  if word in synonyms:
    synonyms.remove(word)

  return list(synonyms) # 집합은 리스트화해서 반환한다.



def synonym_replacement(words, n):

  words = words.split()

  new_words = words.copy()
  random_word_list = list(set([word for word in words if word not in eng_stopwords]))
  random.shuffle(random_word_list)
  num_replaced = 0

  for random_word in random_word_list:
    synonyms = get_synonyms(random_word)

    if len(synonyms) >= 1:
      synonym = random.choice(list(synonyms))
      new_words = [synonym if word == random_word else word for word in new_words]
      num_replaced += 1

    if num_replaced >= n: #only replace up to n words
      break

  sentence = ' '.join(new_words)

  return sentence

In [18]:
# RD

def random_deletion(words, p):
  words = words.split()

  # 하나의 단어로 이루어져 있으면 , 삭제하지 않는다.
  if len(words) == 1 :
    return words

  # 확률 p에 따라, 랜덤하게 단어 삭제

  new_words = []
  for word in words:
    r = random.uniform(0, 1)
    if r > p:
      new_words.append(word)

  # 만약 모든 단어를 삭제했다면, 랜덤한 단어를 리턴한다.
  if len(new_words) == 0:
    rand_int = random.randint(0, len(words) - 1)
    return [words[rand_int]]

  sentence = ' '.join(new_words)

  return sentence

In [19]:
#RS

def swap_word(new_words):

  # 단어를 스왑하는 함수
  random_idx_1 = random.randint(0, len(new_words) - 1)
  random_idx_2 = random_idx_1
  counter = 0

  while random_idx_2 == random_idx_1 :
    random_idx_2 = random.randint(0, len(new_words) -1 )
    counter += 1

    if counter > 3 :
      return new_words

  new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
  return new_words


def random_swap(words, n):

    words = words.split()
    new_words = words.copy()
    # n is the number of words to be swapped
    for _ in range(n):
        new_words = swap_word(new_words)

    sentence = ' '.join(new_words)

    return sentence


In [20]:
# RI

def random_insertion(words, n):

    words = words.split()
    new_words = words.copy()

    for _ in range(n):
        add_word(new_words)

    sentence = ' '.join(new_words)
    return sentence

def add_word(new_words):

    synonyms = []
    counter = 0

    while len(synonyms) < 1:
        random_word = new_words[random.randint(0, len(new_words)-1)]
        synonyms = get_synonyms(random_word)
        counter += 1
        if counter >= 10:
            return

    random_synonym = synonyms[0]
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)

In [21]:
# train 데이터셋을 각각 다음의 작업을 거쳐 데이터셋을 만들어냄.

train['facts_RS'] = train['facts'].apply(lambda x: random_swap(x, 5))
train['facts_RD'] = train['facts'].apply(lambda x: random_deletion(x, 0.5))
train['facts_SR'] = train['facts'].apply(lambda x: synonym_replacement(x, 5))
train['facts_RI'] = train['facts'].apply(lambda x: random_insertion(x, 5))

In [22]:
# train dataset을 열이 아닌 행으로 붙이기 위해, drop을 한 후, rename을 거쳐, 행을 Concatenate한다.
train_1 = train.drop(['facts_RS', 'facts_RD', 'facts_SR', 'facts_RI'], axis=1)
train_2 = train.drop(['facts', 'facts_RD', 'facts_SR', 'facts_RI'], axis=1)
train_3 = train.drop(['facts_RS', 'facts', 'facts_SR', 'facts_RI'], axis=1)
train_4 = train.drop(['facts_RS', 'facts_RD', 'facts', 'facts_RI'], axis=1)
train_5 = train.drop(['facts_RS', 'facts_RD', 'facts_SR', 'facts'], axis=1)

train_2 = train_2.rename(columns={'facts_RS' : 'facts'})
train_3 = train_3.rename(columns={'facts_RD' : 'facts'})
train_4 = train_4.rename(columns={'facts_SR' : 'facts'})
train_5 = train_5.rename(columns={'facts_RI' : 'facts'})

# Concatenate한다.
train_extended = pd.concat([train_1, train_2, train_3, train_4, train_5], axis=0)
train_extended.reset_index(drop=True, inplace=True)
# 즉, 데이터셋은 같은 데이터를 5번 반복한 것이나 다름없다. 다만, 조정을 통해 완전 같은 데이터셋이 아님.


### Train, Test JSON 파일

In [23]:
%cd "/content/drive/MyDrive/LLaMaprac"

/content/drive/MyDrive/LLaMaprac


In [24]:
law_dicts_train = []

for idx, row in train_extended.iterrows():
    law_dict = {'instruction' : '', 'input' : '', 'output' : ''}
    law_dict['instruction'] = f"Who is the winner of below case? The first party is {row['first_party']} and The second party is {row['second_party']}."
    law_dict['input'] = f"{row['facts']}"
    if row['first_party_winner'] == 1:
        law_dict['output'] = f"The winner of this case is {row['first_party']}."
    else:
        law_dict['output'] = f"The winner of this case is {row['second_party']}."

    law_dicts_train.append(law_dict)

# JSON 파일 작성
import json

with open("law_dicts_train.json", "w") as json_file:
    json.dump(law_dicts_train, json_file)

In [25]:
# json 파일이 유효한지 확인해봅니다.
with open('/content/drive/MyDrive/LLaMaprac/law_dicts_train.json') as file:
    try:
        data = json.load(file)
        print("JSON 파일이 유효합니다.")

    except json.JSONDecodeError as e:
        print("JSON 파일이 유효하지 않습니다. 오류 메시지:", e)

JSON 파일이 유효합니다.


In [26]:
# test 파일은 나중에 beautifulsoup이나 selenium 등으로, 학습한 모델의 URL에 접근해서 결과를 얻어올 때 사용하시면 됩니다.
# 필요하지 않다면 test 파일은 json으로 안만드셔도 무방합니다.

law_dicts_test = []

for idx, row in test.iterrows():
    law_dict = {'instruction' : '', 'input' : ''}
    law_dict['instruction'] = f"Who is the winner of below case? The first party is {row['first_party']} and The second party is {row['second_party']}."
    law_dict['input'] = f"{row['facts']}"

    law_dicts_test.append(law_dict)

import json

with open("law_dicts_test.json", "w") as json_file:
    json.dump(law_dicts_test, json_file)

In [27]:
# Test json 파일 유효성 확인
with open('/content/drive/MyDrive/LLaMaprac/law_dicts_test.json') as file:
    try:
        data_test = json.load(file)
        print("JSON 파일이 유효합니다.")

    except json.JSONDecodeError as e:
        print("JSON 파일이 유효하지 않습니다. 오류 메시지:", e)

JSON 파일이 유효합니다.


In [28]:
# GPT, LLama, Alpaca 등 LLM 모델에 필요한 template 파일을 정의하는 코드입니다.
# 아래 코드는 Alpaca-Lora가 인식하는 template의 형식입니다. 아래처럼 진행해서, 마찬가지로 json 파일로 저장해주시면 됩니다.

prompt_template = {
    "description" : "Alpaca-Lora Custom Template",
    "prompt_input" : (
        "Below is an instruction that describes a task, Paired with an input that provides further context.\n"
        "Writhe a response that appropriately completes the request.\n"
        "### Instruction : \n{instruction}\n\n### Input : \n{input}\n\n### Response:\n"
    ),
    "prompt_no_input" : (
        "Below is an instruction that describes a task.\n"
        "Writhe a response that appropriately completes the request.\n"
        "### Instruction : \n{instruction}\n\n### Input : \n{input}\n\n### Response:\n"
    ),
    "response_split" : "### Response:"
}

with open("/content/drive/MyDrive/LLaMaprac/llama/templates/custom_template.json", "w", encoding='utf-8') as f:
    json.dump(prompt_template, f, ensure_ascii=False)